<a href="https://colab.research.google.com/github/nxxk23/PROJECT/blob/main/FileDiag.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import pandas as pd
hemo = pd.read_csv('/content/drive/MyDrive/Thesis Project Kidney/Hemodialysis.csv', encoding='latin1')

## **DiagSummary**

In [3]:
diagnosis = pd.read_csv('/content/drive/MyDrive/Thesis Project Kidney/Diagnosis.csv')

In [4]:
patient_uid_list = hemo['PatientUID'].unique().tolist()
diagnosis = diagnosis[~diagnosis['PatientUID'].isin(patient_uid_list)]

In [5]:
diagnosis0 = pd.DataFrame(diagnosis)

In [6]:
diag_code = diagnosis['ProblemCode'].value_counts().reset_index()
diag_code.columns = ['UniqueValue_CODE','Count']

diag_name = diagnosis['ProblemName'].value_counts().reset_index()
diag_name.columns = ['UniqueValue_NAME','Count']

# Specify the columns for unique value counting
columns_to_count = ['ProblemCode', 'ProblemName']

# Count unique values in the specified columns
diag_count = diagnosis[columns_to_count].value_counts().reset_index()
diag_count.columns = ['ProblemCode', 'ProblemName', 'Count']

diag_df = pd.DataFrame(diag_count)

In [7]:
DiagSummaryDF = pd.DataFrame(diag_df)

In [8]:
DiagSummaryDF

,ProblemCode,ProblemName,Count
0,370992007,Dyslipidemia,124826
1,I10,Essential (primary) hypertension,90673
2,44054006,Type 2 diabetes mellitus,68737
3,1201005,Benign essential hypertension,66796
4,59621000,Essential hypertension,51307
...,...,...,...
18914,31137003,"Early latent syphilis, positive serology, nega...",1
18915,311804006,Prolapsed lumbar intervertebral disc with scia...,1
18916,125587004,Superficial injury,1
18917,311892004,Collapse of vertebra due to osteoporosis NOS,1


## **Slope**
>> **ค่า x = ค่าจากวิสิท t ที่มาก่อน**

In [9]:
import pandas as pd
slope = pd.read_excel('/content/drive/MyDrive/Thesis Project Kidney/Assign/SummaryFile/LabSlope.xlsx')

In [10]:
data = pd.DataFrame(slope)
data = data[['PatientUID','PatientVisitUID','Slopes']]
data = data.dropna()

In [11]:
df = pd.DataFrame(data)

# Sort the DataFrame by PatientUID and PatientVisitUID
df = df.sort_values(by=['PatientUID', 'PatientVisitUID'])

# Create a new column to store the previous slope value
df['PreviousSlope'] = df['Slopes'].shift(1)

# Update the Slopes column with the PreviousSlope values
df['Slopes'] = df['PreviousSlope']

# Drop the PreviousSlope column
df = df.drop(columns=['PreviousSlope'])
data = df.dropna()

In [12]:
data

,PatientUID,PatientVisitUID,Slopes
2,2299,4286612,0.002857
3,2299,4546471,0.020833
4,2299,4801938,-0.062921
5,2299,5051781,-0.056522
6,2299,5296557,0.047619
...,...,...,...
280078,3409733,10435209,-0.655833
280079,3432056,10235244,-0.208313
280080,3432056,10306813,0.168400
280081,3432056,10408954,1.129412


# **DiagnosisList**

In [13]:
mapping = {
    'diag1': [1,3,4,10,81,395,510,1148,1668,2138,2318,2519,3269,3824,4084,4432,4534,5399,5853,6060,6913,7518,8336,8359,9084,10694,10869,12685,13526,13665,13781,15605,16508,16781,16900,16938,17271,17363],
    'diag2': [2,5,7,20,41,69,116,178,181,184,195,206,275,279,495,514,520,650,694,710,753,825,918,975,1061,1099,1142,1197,1242,1266,1389,1435,1466,1496,1509,1544,1585,1715,1723,1865,2095,2383,3186,3266,3284,3361,3413,3564,4004,4096,4474,4655,4719,5396,5541,5587,5770,5830,5856,5999,6059,6062,6248,6358,6374,6390,6391,6756,6833,6915,6916,7453,7568,7650,7986,8360,8402,8590,8633,8735,8983,9121,9303,9390,9739,9761,10082,10168,10558,10573,10577,10954,10977,10998,11772,12180,12651,12837,13020,13089,13417,13646,13916,14137,14907,14909,14911,14953,14958,15131,15282,16089,16238,16493,16762,17119,17157,18197,18809,18880,18881,18899,18902,18903,18904],
    'diag3': [0,514,17157,29,420,998,4353,6655,8189,8409,14889,14970,15011],
    'diag4': [288,375,428,1034,1562,7183,8062,54,109,416,1033,1413,1993,3929,4316,4987,5486,6754,7078,7667,11832],
    'diag5': [160,225,374,564,654,870,982,1114,2866,6424,9210,9879,13665,14495,18491],
    'diag6': [53,67,129,257,517,989,1066,1139,1385,1491,1641,1958,2010,2229,2286,2341,2435,3049,3184,3213,3285,3508,3812,4011,4170,4201,4378,4571,4790,4924,5212,5300,5914,5931,6236,6251,6262,6292,6614,6912,7849,7852,7998,8533,9300,9308,9383,10165,10187,10607,11070,11082,11084,11465,11671,11888,11911,12509,12542,12575,12580,12701,13381,13581,13998,14004,14173,14216,14218,14241,14254,14264,14270,14999],
    'diag7': [155,1438,1650,2062,2958,3084,3255,3618,3690,4229,4834,5348,5450,5590,6435,6790,7112,7530,9208,9400,10785,10892,12365,12370,12372,13470,13735,15508,18402,18403,18406,18407,18410,3398,3618,3851,3995,4200,4366,4376,6053,7153,8683,8755,9741,10013,11372,11401,12135,13748,14277,16026,16030,16086,16095,17151,17872,18349,18434,18754],
    'diag8': [32,196,528,1819,2108,2357,2705,7695,14483],
    'diag9': [42,44,534,1967,3456,11415,17132],
    'diag10': [211,887,1741,2000,3418,7013,9527,80,153,220,409,574,625,839,1401,1439,1771,1913,2335,3300,3423,3436,3595,3765,3877,3938,4174,4239,4939,5426,7927,8417,8559,8630,8883,8910,9559,9801,9885,11288,11986,12931,13200,15930,16146,17451,17929,18284],
    'diag11': [188,398,404,448,569,970,1011,1298,1503,2065,2354,2512,3232,3443,4812,4900,5367,5582,6001,7539,8010,8049,8447,8682,9176,10352,10601,10670,10745,13304,13404,13425,13442,13482,14717,16164,16165,16643,16741,17108,17217,17636,17842,18196]
}

# Create the mapping of diagnosis DataFrames
diagnosis_mapping = {}
for diagnosis_name, indices in mapping.items():
    diagnosis_mapping[diagnosis_name] = DiagSummaryDF[DiagSummaryDF.index.isin(diagnosis.index[indices])]

# Create a new DataFrame to store the mapping values
mapping_data = []
for diagnosis_name, diag_df in diagnosis_mapping.items():
    result_item_names = diag_df['ProblemName'].tolist()
    mapping_data.extend([(diagnosis_name, name) for name in result_item_names])

mapping_df = pd.DataFrame(mapping_data, columns=['MappingValue', 'ProblemName'])

# Merge the mapping_df with the original diagnosis DataFrame based on 'ProblemName'
diagnosis = diagnosis.merge(mapping_df, on='ProblemName', how='left')


In [14]:
diagnosis = diagnosis[diagnosis['MappingValue'].notnull()]

In [15]:
diagnosis.head()

,PatientUID,PatientVisitUID,ProblemCode,ProblemName,ProblemDescription,DIAGTYPE,OnsetDttm,MappingValue
4,22234,0,E789,"Disorder of lipoprotein metabolism\,unspecified",ความผิดปกติแบบอื่นของเมตะบอลิซึมของไลโปโปรตีน ...,PRINCIPAL DX (การวินิจฉัยโรคหลัก),2556-01-07 00:00:00.000,diag2
6,22234,0,H811,Benign paroxysmal vertigo,การรู้สึกหมุนเป็นระยะแบบไม่รุนแรง,PRINCIPAL DX (การวินิจฉัยโรคหลัก),2556-01-07 00:00:00.000,diag2
7,22234,0,H811,Benign paroxysmal vertigo,การรู้สึกหมุนเป็นระยะแบบไม่รุนแรง,PRINCIPAL DX (การวินิจฉัยโรคหลัก),2556-01-07 00:00:00.000,diag3
8,17353,0,M329,"Systemic lupus erythematosus\, unspecified",ลูปัส อีริทิมาโตซัส ทั่วร่าง ไม่ระบุรายละเอียด,PRINCIPAL DX (การวินิจฉัยโรคหลัก),2556-01-07 00:00:00.000,diag11
17,20936,0,E789,"Disorder of lipoprotein metabolism\,unspecified",ความผิดปกติแบบอื่นของเมตะบอลิซึมของไลโปโปรตีน ...,PRINCIPAL DX (การวินิจฉัยโรคหลัก),2556-01-03 00:00:00.000,diag2


In [16]:
diagnosis.shape

(605496, 8)

In [17]:
diagnosis['MappingValue'].nunique()

11

In [18]:
diagnosis['PatientUID'].nunique()

16966

In [19]:
diagnosis['PatientUID'].nunique()

16966

# **DiagnosisTable**
> **จำนวนการเข้ารับการตรวจ**

In [20]:
import pandas as pd

# Group by PatientUID and PatientVisitUID and count each diagnosis
diagnosis_counts = diagnosis.groupby(['PatientUID', 'PatientVisitUID', 'MappingValue']).size().unstack(fill_value=0)

# Rename the columns to Diag1, Diag2, ..., Diag11
diagnosis_counts.columns = ['Diag{}'.format(col.split('diag')[1]) for col in diagnosis_counts.columns]

# Merge the diagnosis counts back into the data table
data = pd.merge(data, diagnosis_counts, how='left', on=['PatientUID', 'PatientVisitUID'])

# Fill NaN values with 0
data.fillna(0, inplace=True)


In [21]:
diagnosis[diagnosis['PatientVisitUID']==4546471]

,PatientUID,PatientVisitUID,ProblemCode,ProblemName,ProblemDescription,DIAGTYPE,OnsetDttm,MappingValue
34645,2299,4546471,370992007,Dyslipidemia,NaN,CO-MORBIDITY(การวินิจฉัยโรคร่วม),2560-10-03 00:00:00.000,diag3
34646,2299,4546471,1201005,Benign essential hypertension,NaN,CO-MORBIDITY(การวินิจฉัยโรคร่วม),2560-10-03 00:00:00.000,diag2
34647,2299,4546471,239873007,Osteoarthritis of knee,NaN,PRINCIPAL DX (การวินิจฉัยโรคหลัก),2560-10-03 00:00:00.000,diag2


In [22]:
data

,PatientUID,PatientVisitUID,Slopes,Diag1,Diag10,Diag11,Diag2,Diag3,Diag4,Diag5,Diag6,Diag7,Diag8,Diag9
0,2299,4286612,0.002857,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2299,4546471,0.020833,0.0,0.0,0.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2299,4801938,-0.062921,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2299,5051781,-0.056522,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2299,5296557,0.047619,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
280076,3409733,10435209,-0.655833,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
280077,3432056,10235244,-0.208313,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
280078,3432056,10306813,0.168400,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
280079,3432056,10408954,1.129412,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### **RenameColumn**

In [23]:
DiagDF = pd.DataFrame(data)

In [24]:
column_rename_dict = {
    'Diag1': 'D1:Hypertension',
    'Diag2': 'D2:Diabetes millitus',
    'Diag3': 'D3:Dyslipidemia',
    'Diag4': 'D4:SLE',
    'Diag5': 'D5:Systemic sclerosis',
    'Diag6': 'D6:Rheumatoid arthritis',
    'Diag7': 'D7:HIV infection',
    'Diag8': 'D8:HBV infection',
    'Diag9': 'D9:HCV infection',
    'Diag10': 'D10:Ischemic heart disease',
    'Diag11': 'D11:Heart failure'
}
# Use the rename method to rename the columns
DiagDF.rename(columns=column_rename_dict, inplace=True)
DiagDF

,PatientUID,PatientVisitUID,Slopes,D1:Hypertension,D10:Ischemic heart disease,D11:Heart failure,D2:Diabetes millitus,D3:Dyslipidemia,D4:SLE,D5:Systemic sclerosis,D6:Rheumatoid arthritis,D7:HIV infection,D8:HBV infection,D9:HCV infection
0,2299,4286612,0.002857,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2299,4546471,0.020833,0.0,0.0,0.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2299,4801938,-0.062921,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2299,5051781,-0.056522,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2299,5296557,0.047619,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
280076,3409733,10435209,-0.655833,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
280077,3432056,10235244,-0.208313,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
280078,3432056,10306813,0.168400,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
280079,3432056,10408954,1.129412,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [26]:
DiagDF = DiagDF[['PatientUID','PatientVisitUID','Slopes','D1:Hypertension','D2:Diabetes millitus','D3:Dyslipidemia',
        'D4:SLE','D5:Systemic sclerosis','D6:Rheumatoid arthritis','D7:HIV infection','D8:HBV infection',
        'D9:HCV infection','D10:Ischemic heart disease','D11:Heart failure']]

In [ ]:
DiagDF.to_excel('DiagDF.xlsx')